# Living Map 疫情地图

## 技术栈
- 地图选择
- 疫情数据采集
- Sqlite存储
- html展示
    - 地图可视化
    - 定位
    - 搜索

## 地图选择

> 选择高德地图，百度地图申请太麻烦

[参考](https://blog.csdn.net/qq_41103204/article/details/105707546)

In [470]:
import requests
import json

def query(**kwargs):
    url = 'http://restapi.amap.com/v3/geocode/geo?parameters'
    params = {
        'city': '上海市',
        'key': 'your key',
        'output': 'json'}
    for k, v in kwargs.items():
        params[k] = v
    res = requests.get(url,params)
    return json.loads(res.content)

In [471]:
query(address='莲花路')

DEBUG 2022-04-19 10:24:48,406 [connectionpool.py:225]: Starting new HTTP connection (1): restapi.amap.com:80
DEBUG 2022-04-19 10:24:48,467 [connectionpool.py:437]: http://restapi.amap.com:80 "GET /v3/geocode/geo?parameters&city=%E4%B8%8A%E6%B5%B7%E5%B8%82&key=your+key&output=json&address=%E8%8E%B2%E8%8A%B1%E8%B7%AF HTTP/1.1" 200 None


{'status': '0', 'info': 'INVALID_USER_KEY', 'infocode': '10001'}

#### html内嵌地图

[参考](https://blog.csdn.net/qq_42445490/article/details/89155610)

[官方地图](https://lbs.amap.com/tools/picker)

[官方手册](https://lbs.amap.com/api/lightmap/gettingstarted)

[官方例子](https://lbs.amap.com/demo/javascript-api/example/)

<html>
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="initial-scale=1.0, user-scalable=no, width=device-width">
    <link rel="stylesheet" href="https://a.amap.com/jsapi_demos/static/demo-center/css/demo-center.css"/>
    <title>地图显示</title>
    <style>
        html,
        body,
        #container {
          width: 100%;
          height: 100%;
        }

    </style>
</head>
<body>
<div id="container"></div>
<!-- 加载地图JSAPI脚本 -->
<script src="https://webapi.amap.com/maps?v=1.4.15&key=您申请的key值"></script>
<script>
    var marker, map = new AMap.Map('container', {
        resizeEnable: true,
        zoom:11,
        center: [121.473667,31.230525]
    });

    marker = new AMap.Marker({
            icon: "//a.amap.com/jsapi_demos/static/demo-center/icons/poi-marker-default.png",
            position: [121.473667,31.230525],
            offset: new AMap.Pixel(-13, -30)
    });
    marker.setMap(map);

</script>
</body>
</html>

<iframe src="data/research/living_map.html" title="Living Map"></iframe>

[Living Map](data/research/living_map.html)

## 疫情数据采集

[上海卫健委](https://wsjkw.sh.gov.cn/yqtb/)

#### 参考网址

[https://chenfan.info/](https://chenfan.info/)

In [124]:
!cd crawler; scrapy crawl living-spider

DEBUG 2022-04-14 00:28:18,548 [db.py:114]: select * from district
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,549 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-14 00:28:18,550 [db.py:114]: PRA

## 上海卫检网数据采集

In [125]:
import requests

def collect(url, **params):
    headers ={
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    }
    params = {}
    r = requests.get(url, params = params, headers = headers)
    return r

In [126]:
import re
def match(pattern, s):
    return re.findall(pattern, s)

In [127]:
url = 'https://wsjkw.sh.gov.cn/yqtb/'

In [128]:
r = collect(url)

DEBUG 2022-04-14 00:28:24,350 [connectionpool.py:959]: Starting new HTTPS connection (1): wsjkw.sh.gov.cn:443
DEBUG 2022-04-14 00:28:24,492 [connectionpool.py:437]: https://wsjkw.sh.gov.cn:443 "GET /yqtb/ HTTP/1.1" 200 None


In [129]:
main_html = r.text

In [130]:
from bs4 import BeautifulSoup
from lxml import etree

#### 采集居住地链接

In [131]:
main_soup = BeautifulSoup(main_html, "html.parser")
main_dom = etree.HTML(str(main_soup))

In [132]:
main_contents = main_dom.xpath('//ul[@class="uli16 nowrapli list-date"]/li')

In [133]:
class Link:
    def __init__(self, month, day, href):
        self.month = month
        self.day = day
        self.href = href
        
    def __repr__(self):
        return str(self.__dict__)

links = []
for content in main_contents:
    href = content.xpath('.//@href')[0]
    title = content.xpath('.//@title')[0]
    pattern = r'(\d+)月(\d+)日.*本市各区确诊病例、无症状感染者居住地信息$'
    groups = match(pattern, title)
    if groups:
        month, day = groups[0]
        links.append(Link(month, day, href))
    
links

[{'month': '4', 'day': '12', 'href': 'https://mp.weixin.qq.com/s/OZGM-pNkefZqWr0IFRJj1g'},
 {'month': '4', 'day': '11', 'href': 'https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA'},
 {'month': '4', 'day': '10', 'href': 'https://mp.weixin.qq.com/s/u0XfHF8dgfEp8vGjRtcwXA'},
 {'month': '4', 'day': '9', 'href': 'https://mp.weixin.qq.com/s/_Je5_5_HqBcs5chvH5SFfA'},
 {'month': '4', 'day': '8', 'href': 'https://mp.weixin.qq.com/s/79NsKhMHbg09Y0xaybTXjA'},
 {'month': '4', 'day': '7', 'href': 'https://mp.weixin.qq.com/s/HTM47mUp0GF-tWXkPeZJlg'},
 {'month': '4', 'day': '6', 'href': 'https://mp.weixin.qq.com/s/8bljTUplPh1q4MXb6wd_gg'},
 {'month': '4', 'day': '5', 'href': 'https://mp.weixin.qq.com/s/djwW3S9FUYBE2L5Hj94a3A'},
 {'month': '4', 'day': '4', 'href': '/xwfb/20220405/4c6aec72ef47453ba2a5643fad214b2a.html'},
 {'month': '4', 'day': '3', 'href': '/xwfb/20220404/ff41c17c2bec4154b800f22040d3754a.html'}]

#### 采集居住地信息

In [134]:
resident_res = collect(links[0].href)
resident_res

DEBUG 2022-04-14 00:28:34,796 [connectionpool.py:959]: Starting new HTTPS connection (1): mp.weixin.qq.com:443
DEBUG 2022-04-14 00:28:34,949 [connectionpool.py:437]: https://mp.weixin.qq.com:443 "GET /s/OZGM-pNkefZqWr0IFRJj1g HTTP/1.1" 200 87230


<Response [200]>

In [135]:
resident_html = resident_res.text
resident_soup = BeautifulSoup(resident_html, 'html.parser')
resident_dom = etree.HTML(str(resident_soup))

In [136]:
section_data_list = resident_dom.xpath('//div[@id="js_content"]/section[@data-id and not(@data-role)]')
len(section_data_list)

16

In [137]:
for i, section_data in enumerate(section_data_list):
    data_id = section_data.xpath('.//@data-id')[0]
    print(f'{i}: {data_id}')

0: 106156
1: 72469
2: 72469
3: 72469
4: 72469
5: 72469
6: 72469
7: 72469
8: 72469
9: 72469
10: 72469
11: 72469
12: 72469
13: 72469
14: 72469
15: 72469


In [141]:
main_section = section_data_list[0]
main_section_data_id = main_section.xpath('.//@data-id')[0]
main_section_data_id

'106156'

In [142]:
main_section_data = main_section.xpath('.//section[@data-id and not(@data-role)]')
main_section_data

[<Element section at 0x7fea2bc82c08>, <Element section at 0x7fea2bca0a88>]

In [146]:
main_section_data[0].xpath('.//p/span')[0].text

'市卫健委今早（13日）通报：2022年4月12日0—24时，新增本土新冠肺炎确诊病例1189例和无症状感染者25141例，其中23例确诊病例为此前无症状感染者转归，867例确诊病例和24500例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。'

In [147]:
main_section_data[0].xpath('.//p/span')[1].text

'各区信息如下：'

In [149]:
main_section_text_list = []
for i, data in enumerate(main_section_data):
    data_id = data.xpath('.//@data-id')[0]
    print(data_id)
    span_list = data.xpath('.//p/span')
    section_text_list = []
    for span in span_list:
        section_text_list.append(span.text)
    main_section_text_list.append(section_text_list)

106156
72469


In [151]:
main_section_text_list[0]

['市卫健委今早（13日）通报：2022年4月12日0—24时，新增本土新冠肺炎确诊病例1189例和无症状感染者25141例，其中23例确诊病例为此前无症状感染者转归，867例确诊病例和24500例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。',
 '各区信息如下：']

In [519]:
other_section_text_list = []
for data in section_data_list[1:]:
    data_id = data.xpath('.//@data-id')[0]
    print(data_id)
    span_list = data.xpath('.//p/span')
    section_text_list = []
    for span in span_list:
        section_text_list.append(span.text)
    other_section_text_list.append(section_text_list)

len(other_section_text_list)

In [206]:
class Resident:
    def __init__(self, **kwargs):
        self.published_date =  kwargs['published_date '] if 'published_date' in kwargs else None
        self.district =  kwargs['district '] if 'district' in kwargs else ''
        self.diagnosed =  kwargs['diagnosed '] if 'diagnosed' in kwargs else 0
        self.asymptomatic =  kwargs['asymptomatic '] if 'asymptomatic' in kwargs else 0
        self.address =  kwargs['address '] if 'address' in kwargs else 0
        self.children = []
        
    def __repr__(self):
        return str(self.__dict__)

In [ ]:
def collect_resident(html):
    soup = BeautifulSoup(html, "html.parser")

## 数据库设计

> name: living_map

![living_map_db](./living_map_db.jpg)

#### 建立 resident_link表

In [530]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "create resident link"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "create resident link"}')
**********************************************
*   alter_living_map: create resident link   *
**********************************************
alembic revision -m "create resident link table"
DEBUG 2022-04-13 13:57:25,143 [header.py:143]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/56dd47ac875b_create_resident_link_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [532]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-13 14:01:51,622 [header.py:143]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-13 14:01:52,004 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-13 14:01:52,004 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-13 14:01:52,006 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-13 14:01:52,006 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-13 1

#### 建立 resident表

In [533]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "create resident"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "create resident"}')
*****************************************
*   alter_living_map: create resident   *
*****************************************
alembic revision -m "create resident table"
DEBUG 2022-04-13 14:03:04,861 [header.py:143]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/f7652f427aee_create_resident_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [534]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-13 14:15:42,905 [header.py:143]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-13 14:15:43,319 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-13 14:15:43,320 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-13 14:15:43,321 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-13 14:15:43,321 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-13 1

#### 建立resident_summary表

In [535]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "create resident summary"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "create resident summary"}')
*************************************************
*   alter_living_map: create resident summary   *
*************************************************
alembic revision -m "create resident summary table"
DEBUG 2022-04-13 14:16:35,395 [header.py:143]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/bdf577ec62b0_create_resident_summary_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [536]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-13 14:20:52,789 [header.py:143]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-13 14:20:53,203 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-13 14:20:53,203 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-13 14:20:53,205 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-13 14:20:53,205 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()


#### 建立district表

In [537]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "create district"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "create district"}')
*****************************************
*   alter_living_map: create district   *
*****************************************
alembic revision -m "create district table"
DEBUG 2022-04-13 14:22:45,091 [header.py:143]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/8cfd220cc448_create_district_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [538]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-13 14:24:51,840 [header.py:143]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-13 14:24:52,215 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-13 14:24:52,216 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-13 14:24:52,217 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-13 14:24:52,217 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-13 1

#### 初始化district数据

In [539]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "alter district initialise"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "alter district initialise"}')
***************************************************
*   alter_living_map: alter district initialise   *
***************************************************
alembic revision -m "alter district initialise table"
DEBUG 2022-04-13 14:26:01,354 [header.py:143]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/a4835b378b13_alter_district_initialise_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [158]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-14 14:29:40,575 [header.py:145]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-14 14:29:41,076 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-14 14:29:41,076 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-14 14:29:41,078 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-14 14:29:41,078 INFO sqlalchemy.engine.Engine [generated in 0.00018s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00018s] ()
2022-04-14 1

In [157]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "alter  resident_link add origin_path"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "alter  resident_link add origin_path"}')
**************************************************************
*   alter_living_map: alter  resident_link add origin_path   *
**************************************************************
alembic revision -m "alter  resident_link add origin_path table"
DEBUG 2022-04-14 14:27:42,727 [header.py:145]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/b3fe734c4531_alter_resident_link_add_origin_path_.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


#### 在resident_link增加origin_filepath

In [159]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-14 14:29:50,556 [header.py:145]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-14 14:29:50,941 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-14 14:29:50,941 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-14 14:29:50,942 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-14 14:29:50,942 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00012s] ()


#### 建立 resident_input表

In [160]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "create resident input"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "create resident input"}')
***********************************************
*   alter_living_map: create resident input   *
***********************************************
alembic revision -m "create resident input table"
DEBUG 2022-04-14 14:33:28,364 [header.py:145]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/497d47b29858_create_resident_input_table.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [161]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-14 14:37:02,987 [header.py:145]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-14 14:37:03,508 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-14 14:37:03,508 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-14 14:37:03,509 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-14 14:37:03,509 INFO sqlalchemy.engine.Engine [generated in 0.00022s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00022s] ()
2022-04-14 1

#### 修改resident_summary 增加origin_desc

In [453]:
!python3 service.py scheme update -p '{"db":"living_map", "build": "alter resident summary add origin desc"}'

Namespace(action='update', name='scheme', p='{"db":"living_map", "build": "alter resident summary add orgin desc"}')
***************************************************************
*   alter_living_map: alter resident summary add orgin desc   *
***************************************************************
alembic revision -m "alter resident summary add orgin desc table"
DEBUG 2022-04-18 15:44:59,406 [header.py:164]: run at "db/living_map"
  Generating /Users/yixiaobin/local/workspace/living_manual_at_2022/db/living_map/
  scheme/versions/c54ed564c672_alter_resident_summary_add_orgin_desc_.py ...  done
******************************************************************
*   generate file under db/living_map/scheme/versions, edit it   *
******************************************************************


In [454]:
!python3 service.py scheme update -p '{"db":"living_map"}'

Namespace(action='update', name='scheme', p='{"db":"living_map"}')
*************************
*   update_living_map   *
*************************
alembic upgrade head
DEBUG 2022-04-18 15:46:57,669 [header.py:164]: run at "db/living_map"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022-04-18 15:46:58,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_info("alembic_version")
2022-04-18 15:46:58,212 INFO sqlalchemy.engine.Engine [raw sql] ()
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
2022-04-18 15:46:58,214 INFO sqlalchemy.engine.Engine SELECT alembic_version.version_num
FROM alembic_version
INFO  [sqlalchemy.engine.Engine] SELECT alembic_version.version_num
FROM alembic_version
2022-04-18 15:46:58,214 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ()
INFO  [sqlalchemy.engine.Engine] [generated in 0.00013s] ()
2022-04-18 1

## 采集入库

In [186]:
import os
import sys
import logging
import numpy as np
import pandas as pd

FORMAT = '%(levelname)s %(asctime)s [%(filename)s:%(lineno)d]: %(message)s'
logging.basicConfig(format=FORMAT, stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()

In [381]:
import os
import sqlite3

class DB:
    def __init__(self):
        self.db_con = sqlite3.connect(os.path.join('db', 'living_map', 'living_map.db'))
        

    def find(self, table_name):
        return self._select_(f'select * from {table_name}')

    def get(self, table_name, id):
        statement = (f'select * from {table_name} where id = ?', (id,))
        dataset = self._select_(statement)
        if len(dataset) == 1:
            return dataset[0]
        return None

    def query(self, sql):
        return self._select_(sql)

    def create(self, table_name, **scheme):
        sqls = []
        sqls.append(f'DROP TABLE IF EXISTS {table_name}')
        fields = []
        for n, t in scheme.items():
            fields.append(f'{n} t')
        fields = ','.join(fields)
        sqls.append(f'create table {table_name}({fields})')
        self._udpate_(sqls)

    def insert(self, table_name, **fields):
        names = []
        values = []
        for k, v in fields.items():
            names.append(k)
            values.append(v)
        names = ','.join(names)
        if len(fields.items()) == 1:
            refs = '?'
        else:
            refs = ','.join('?' for _ in range(len(fields.items())))
        sql = f'insert into {table_name}({names}) values({refs})'
        statement = (sql, values)
        self._udpate_([statement])

    def delete(self, table_name):
        sql = f'delete from {table_name}'
        self._udpate_([sql])

    def drop(self, table_name):
        sql = f'drop table if exists {table_name}'
        self._udpate_([sql])

    def update(self, sql):
        self._udpate_([sql])

    def _udpate_(self, sqls):
        db_cur = self.db_con.cursor()
        for sql in sqls:
            logger.debug(sql)
            if type(sql) is tuple:
                db_cur.execute(sql[0], sql[1])
            else:
                db_cur.execute(sql)
        self.db_con.commit()

    def _select_(self, sql):
        db_cur = self.db_con.cursor()
        if type(sql) is tuple:
            db_cur.execute(sql[0], sql[1])
        else:
            db_cur.execute(sql)
        dataset = db_cur.fetchall()
        db_cur.close()
        return dataset
    
db = DB()

#### 采集resident link

In [391]:
db.drop('resident')

DEBUG 2022-04-16 16:43:00,632 [<ipython-input-381-d5828c340204>:61]: drop table if exists resident


In [392]:
db._udpate_(['''
CREATE TABLE resident (
    id INTEGER NOT NULL, 
    published_at DATETIME, 
    origin VARCHAR, 
    address VARCHAR, 
    longitude FLOAT, 
    latitude FLOAT, 
    diagnosed INTEGER,
    asymptomatic INTEGER, 
    link_id INTEGER,
    summary_id INTEGER,
    district_code VARCHAR, 
    created_at DATETIME, 
    updated_at DATETIME, 
    state INTEGER, 
    PRIMARY KEY (id)
)      
'''])

DEBUG 2022-04-16 16:43:01,928 [<ipython-input-381-d5828c340204>:61]: 
CREATE TABLE resident (
    id INTEGER NOT NULL, 
    published_at DATETIME, 
    origin VARCHAR, 
    address VARCHAR, 
    longitude FLOAT, 
    latitude FLOAT, 
    diagnosed INTEGER,
    asymptomatic INTEGER, 
    link_id INTEGER,
    summary_id INTEGER,
    district_code VARCHAR, 
    created_at DATETIME, 
    updated_at DATETIME, 
    state INTEGER, 
    PRIMARY KEY (id)
)      



In [393]:
db.query('SELECT * FROM sqlite_master WHERE type="table"')

[('table',
  'alembic_version',
  'alembic_version',
  2,
  'CREATE TABLE alembic_version (\n\tversion_num VARCHAR(32) NOT NULL, \n\tCONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)\n)'),
 ('table',
  'resident_link',
  'resident_link',
  4,
  'CREATE TABLE resident_link (\n\tid INTEGER NOT NULL, \n\tpublished_at DATETIME, \n\torigin VARCHAR, \n\tlink VARCHAR, \n\tcreated_at DATETIME, \n\tupdated_at DATETIME, \n\tstate INTEGER, origin_filepath char(255), \n\tPRIMARY KEY (id)\n)'),
 ('table',
  'resident_summary',
  'resident_summary',
  6,
  'CREATE TABLE resident_summary (\n\tid INTEGER NOT NULL, \n\tpublished_at DATETIME, \n\torigin VARCHAR, \n\tdiagnosed INTEGER, \n\tasymptomatic INTEGER, \n\tdistrict_code VARCHAR, \n\tcreated_at DATETIME, \n\tupdated_at DATETIME, \n\tstate INTEGER, \n\tPRIMARY KEY (id)\n)'),
 ('table',
  'district',
  'district',
  7,
  'CREATE TABLE district (\n\tid INTEGER NOT NULL, \n\tname VARCHAR, \n\tcode VARCHAR, \n\tcreated_at DATETIME, \n\tupdated_

In [154]:
db.delete('resident_link')
db.query('select * from resident_link')

DEBUG 2022-04-14 14:18:44,568 [<ipython-input-31-d5828c340204>:61]: delete from resident_link


[]

In [376]:
db.query('select * from resident_link')

[(1,
  '2022-04-13 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/L9AffT-SoEBV4puBa_mRqg',
  '2022-04-14 17:02:34.777149',
  '2022-04-14 17:02:34.777149',
  3,
  'mp.weixin.qq.com_s_L9AffT-SoEBV4puBa_mRqg.html'),
 (2,
  '2022-04-12 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/OZGM-pNkefZqWr0IFRJj1g',
  '2022-04-14 17:02:34.779074',
  '2022-04-14 17:02:34.779074',
  3,
  'mp.weixin.qq.com_s_OZGM-pNkefZqWr0IFRJj1g.html'),
 (3,
  '2022-04-11 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA',
  '2022-04-14 17:02:34.780409',
  '2022-04-14 17:02:34.780409',
  3,
  'mp.weixin.qq.com_s_vxFiV2HeSvByINUlTmFKZA.html'),
 (4,
  '2022-04-10 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/u0XfHF8dgfEp8vGjRtcwXA',
  '2022-04-14 17:02:34.781984',
  '2022-04-14 17:02:34.781984',
  3,
  'mp.weixin.qq.com_s_u0XfHF8dgfEp8vGjRtcwXA.

In [111]:
!scrapy crawl resident-link-spider

DEBUG 2022-04-13 21:57:09,188 [db.py:114]: select * from district
DEBUG 2022-04-13 21:57:09,188 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,189 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 21:57:09,190 [db.py:114]: PRA

#### 采集resident

In [112]:
db.query('select link from resident_link')

[('https://mp.weixin.qq.com/s/OZGM-pNkefZqWr0IFRJj1g',),
 ('https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA',),
 ('https://mp.weixin.qq.com/s/u0XfHF8dgfEp8vGjRtcwXA',),
 ('https://mp.weixin.qq.com/s/_Je5_5_HqBcs5chvH5SFfA',),
 ('https://mp.weixin.qq.com/s/79NsKhMHbg09Y0xaybTXjA',),
 ('https://mp.weixin.qq.com/s/HTM47mUp0GF-tWXkPeZJlg',),
 ('https://mp.weixin.qq.com/s/8bljTUplPh1q4MXb6wd_gg',),
 ('https://mp.weixin.qq.com/s/djwW3S9FUYBE2L5Hj94a3A',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220405/4c6aec72ef47453ba2a5643fad214b2a.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220404/ff41c17c2bec4154b800f22040d3754a.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220402/e4dfcc8e58b14b3e8dab46d60ff6d767.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220401/8c101d231d5644df8ed92d6bdbfab236.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220331/e86a298aecdf4e94b1796089e943054b.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220330/8d4f5179f1ef43da91c2d63cd906bfeb.html',),
 ('https://wsjkw.sh.gov.cn/xwfb/20220329/b7430

> 分别对weinxin和wsjkw.sh.gov.cn解析

In [113]:
!scrapy crawl resident-spider

DEBUG 2022-04-13 22:14:08,848 [db.py:114]: select * from district
DEBUG 2022-04-13 22:14:08,848 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,848 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,849 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,850 [db.py:114]: PRAGMA table_info = 'district'
DEBUG 2022-04-13 22:14:08,850 [db.py:114]: PRA

> 保存html到data/origin下

> 保存解析后的json到data/input下

In [218]:
db.delete('resident_input')

DEBUG 2022-04-14 21:22:20,170 [<ipython-input-187-d5828c340204>:61]: delete from resident_input


In [221]:
db.query('select * from resident_input')

[(1,
  'shanghai_resident_2022-04-13 00:00:00.json',
  1,
  '2022-04-14 21:22:39.419606',
  '2022-04-14 21:22:39.419606',
  1),
 (2,
  'shanghai_resident_2022-04-11 00:00:00.json',
  3,
  '2022-04-14 21:22:40.225123',
  '2022-04-14 21:22:40.225123',
  1),
 (3,
  'shanghai_resident_2022-04-06 00:00:00.json',
  8,
  '2022-04-14 21:22:40.855927',
  '2022-04-14 21:22:40.855927',
  1),
 (4,
  'shanghai_resident_2022-04-10 00:00:00.json',
  4,
  '2022-04-14 21:22:41.582016',
  '2022-04-14 21:22:41.582016',
  1),
 (5,
  'shanghai_resident_2022-04-09 00:00:00.json',
  5,
  '2022-04-14 21:22:42.224793',
  '2022-04-14 21:22:42.224793',
  1),
 (6,
  'shanghai_resident_2022-04-07 00:00:00.json',
  7,
  '2022-04-14 21:22:42.819796',
  '2022-04-14 21:22:42.819796',
  1),
 (7,
  'shanghai_resident_2022-04-12 00:00:00.json',
  2,
  '2022-04-14 21:22:43.561464',
  '2022-04-14 21:22:43.561464',
  1),
 (8,
  'shanghai_resident_2022-04-08 00:00:00.json',
  6,
  '2022-04-14 21:22:44.201702',
  '2022-04-14 

In [220]:
db.update('update resident_link set state=2')

DEBUG 2022-04-14 21:22:23,827 [<ipython-input-187-d5828c340204>:61]: update resident_link set state=2


In [213]:
db.query('select * from resident_link')

[(1,
  '2022-04-13 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/L9AffT-SoEBV4puBa_mRqg',
  '2022-04-14 17:02:34.777149',
  '2022-04-14 17:02:34.777149',
  3,
  ''),
 (2,
  '2022-04-12 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/OZGM-pNkefZqWr0IFRJj1g',
  '2022-04-14 17:02:34.779074',
  '2022-04-14 17:02:34.779074',
  3,
  ''),
 (3,
  '2022-04-11 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA',
  '2022-04-14 17:02:34.780409',
  '2022-04-14 17:02:34.780409',
  3,
  ''),
 (4,
  '2022-04-10 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/u0XfHF8dgfEp8vGjRtcwXA',
  '2022-04-14 17:02:34.781984',
  '2022-04-14 17:02:34.781984',
  3,
  ''),
 (5,
  '2022-04-09 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/_Je5_5_HqBcs5chvH5SFfA',
  '2022-04-14 17:02:34.784165',
  '2022-04-14 

## 抽取 shanghai summary

In [222]:
s1 = '市卫健委今早（6日）通报：2022年4月5日0—24时，新增本土新冠肺炎确诊病例311例和无症状感染者16766例，其中40例确诊病例为此前无症状感染者转归，4例确诊病例和16256例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。新增境外输入性新冠肺炎确诊病例4例和无症状感染者1例，均在闭环管控中发现。'
s1

'市卫健委今早（6日）通报：2022年4月5日0—24时，新增本土新冠肺炎确诊病例311例和无症状感染者16766例，其中40例确诊病例为此前无症状感染者转归，4例确诊病例和16256例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。新增境外输入性新冠肺炎确诊病例4例和无症状感染者1例，均在闭环管控中发现。'

In [224]:
s2 = '市卫健委今早（13日）通报：2022年4月12日0—24时，新增本土新冠肺炎确诊病例1189例和无症状感染者25141例，其中23例确诊病例为此前无症状感染者转归，867例确诊病例和24500例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。'
s2

'市卫健委今早（13日）通报：2022年4月12日0—24时，新增本土新冠肺炎确诊病例1189例和无症状感染者25141例，其中23例确诊病例为此前无症状感染者转归，867例确诊病例和24500例无症状感染者在隔离管控中发现，其余在相关风险人群排查中发现。'

In [226]:
import re
def find(s, pattern):
    return re.findall(pattern, s)

In [315]:
# sh_pattern = r'.+：(\d{4})年(\d{1,2})月(\d{1,2})日0—24时，新增本土新冠肺炎确诊病例(\d+)例和无症状感染者(\d+)例，其中(\d+)例确诊病例为此前无症状感染者转归，(\d+)例确诊病例和(\d+)\w+'
sh_pattern = r'.+：(\d{4})年(\d{1,2})月(\d{1,2})日0—24时，\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+'
find(s1, sh_pattern)

[('2022', '4', '5', '311', '16766', '40', '4', '16256')]

In [253]:
find(s2, sh_pattern)

[('2022', '4', '12', '1189', '25141', '23', '867', '24500')]

In [255]:
s3 = '2022年4月5日，浦东新区新增162例本土确诊病例，新增7983例本土无症状感染者，分别居住于：'
s3

'2022年4月5日，浦东新区新增162例本土确诊病例，新增7983例本土无症状感染者，分别居住于：'

In [261]:
s4 = '2022年4月5日，静安区新增12例本土确诊病例，新增290例本土无症状感染者，分别居住于：'
s4

'2022年4月5日，静安区新增12例本土确诊病例，新增290例本土无症状感染者，分别居住于：'

In [371]:
# d_pattern = '(\d{4})年(\d{1,2})月(\d{1,2})日，(\w+)新增(\d+)例本土确诊病例，新增(\d+)例本土无症状感染者，分别居住于：'
# d_pattern = '(\d{4})年(\d{1,2})月(\d{1,2})日，(.+)[无]?新增(\d+|)例本土.*，[无]?新增(\d+|)\D+'
d_pattern = '(\d{4})年(\d{1,2})月(\d{1,2})日，(.+)[无]?新增(\d+|)\D+[无]?新增(\d+)\D+'
find(s3, d_pattern)

[('2022', '4', '5', '浦东新区', '162', '7983')]

In [372]:
find(s4, d_pattern)

[('2022', '4', '5', '静安区', '12', '290')]

In [373]:
s6 = '2022年4月10日，崇明区无新增本土确诊病例，新增55例新冠肺炎无症状感染者，分别居住于：'
s6

'2022年4月10日，崇明区无新增本土确诊病例，新增55例新冠肺炎无症状感染者，分别居住于：'

In [374]:
# d_pattern = '(\d{4})年(\d{1,2})月(\d{1,2})日，(.+)[无]?新增(\d+|)\D+(\d+)\D+'
find(s6, d_pattern)

[('2022', '4', '10', '崇明区无', '', '55')]

## 通过地址得到经纬度

通过voicing spider[https://lbs.amap.com/tools/picker](https://lbs.amap.com/tools/picker)抽取

存入json文件

## 保存resident到数据库

In [457]:
db.find('resident_summary')

[(1,
  '2022-04-13 00:00:00',
  'shanghai_resident_2022-04-13 00:00:00',
  2573,
  25146,
  '31',
  '2022-04-18 15:51:38.942872',
  '2022-04-18 15:51:38.942872',
  1,
  ''),
 (2,
  '2022-04-13 00:00:00',
  'shanghai_resident_2022-04-13 00:00:00',
  1139,
  13888,
  '310115',
  '2022-04-18 15:51:38.947531',
  '2022-04-18 15:51:38.947531',
  1,
  '2022年4月13日，浦东新区新增1139例本土确诊病例，新增13888例本土无症状感染者，分别居住于：'),
 (3,
  '2022-04-13 00:00:00',
  'shanghai_resident_2022-04-13 00:00:00',
  227,
  1181,
  '310101',
  '2022-04-18 15:51:38.948994',
  '2022-04-18 15:51:38.948994',
  1,
  '2022年4月13日，黄浦区新增227例本土确诊病例，新增1181例本土无症状感染者，分别居住于：'),
 (4,
  '2022-04-13 00:00:00',
  'shanghai_resident_2022-04-13 00:00:00',
  49,
  171,
  '310106',
  '2022-04-18 15:51:38.950371',
  '2022-04-18 15:51:38.950371',
  1,
  '2022年4月13日，静安区新增49例本土确诊病例，新增171例本土无症状感染者，分别居住于：'),
 (5,
  '2022-04-13 00:00:00',
  'shanghai_resident_2022-04-13 00:00:00',
  143,
  1349,
  '310104',
  '2022-04-18 15:51:38.951785',
  '2022-04-18 15:5

In [443]:
db.find('resident_link')

[(1,
  '2022-04-13 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/L9AffT-SoEBV4puBa_mRqg',
  '2022-04-14 17:02:34.777149',
  '2022-04-14 17:02:34.777149',
  3,
  'mp.weixin.qq.com_s_L9AffT-SoEBV4puBa_mRqg.html'),
 (2,
  '2022-04-12 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/OZGM-pNkefZqWr0IFRJj1g',
  '2022-04-14 17:02:34.779074',
  '2022-04-14 17:02:34.779074',
  3,
  'mp.weixin.qq.com_s_OZGM-pNkefZqWr0IFRJj1g.html'),
 (3,
  '2022-04-11 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA',
  '2022-04-14 17:02:34.780409',
  '2022-04-14 17:02:34.780409',
  3,
  'mp.weixin.qq.com_s_vxFiV2HeSvByINUlTmFKZA.html'),
 (4,
  '2022-04-10 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/u0XfHF8dgfEp8vGjRtcwXA',
  '2022-04-14 17:02:34.781984',
  '2022-04-14 17:02:34.781984',
  3,
  'mp.weixin.qq.com_s_u0XfHF8dgfEp8vGjRtcwXA.

In [465]:
len(db.find('resident_summary'))

176

In [464]:
len(db.find('resident'))

56585

In [438]:
db.find('resident_summary')[0]

(1,
 '2022-04-13 00:00:00',
 'shanghai_resident_2022-04-13 00:00:00',
 2573,
 25146,
 '31',
 '2022-04-17 01:04:35.500937',
 '2022-04-17 01:04:35.500937',
 1)

In [469]:
db.query('select * from resident_link order by published_at desc')

[(26,
  '2022-04-17 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/LguiUZj-zxy4xy19WO0_UA',
  '2022-04-18 17:24:04.450490',
  '2022-04-18 17:24:04.450490',
  3,
  ''),
 (27,
  '2022-04-16 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/dRa-PExJr1qkRis88eGCnQ',
  '2022-04-18 17:29:57.747615',
  '2022-04-18 17:29:57.747615',
  3,
  ''),
 (28,
  '2022-04-15 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/ZkhimhWpa92I2EWn3hmd8w',
  '2022-04-18 17:29:57.750569',
  '2022-04-18 17:29:57.750569',
  1,
  ''),
 (29,
  '2022-04-14 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/5T76lht3s6g_KTiIx3XAYw',
  '2022-04-18 17:29:57.752417',
  '2022-04-18 17:29:57.752417',
  3,
  ''),
 (1,
  '2022-04-13 00:00:00',
  'https://wsjkw.sh.gov.cn/yqtb/index.html',
  'https://mp.weixin.qq.com/s/L9AffT-SoEBV4puBa_mRqg',
  '2022-04-14 17:02:34.777149',
  '2022-04

## 报告模板库

- [sheetjs](https://github.com/sheetjs/sheetjs)
- [FileSaver](https://github.com/eligrey/FileSaver.js)
- [panelsnap](https://github.com/guidobouman/panelsnap)
- [高德](https://webapi.amap.com)

## Search Box On Map

[代码参考](https://codepen.io/JaminQuimby/pen/vOXQrL)